In [1]:
# import numpy as np
# import pandas as pd
# from scipy.stats import mannwhitneyu
# from skbio.diversity import beta_diversity
# from skbio.stats.distance import permanova
# from statsmodels.stats.multitest import multipletests
# 
# """设置显示选项"""
# pd.set_option('display.max_columns', None)
# pd.set_option('display.expand_frame_repr', False)
# 
# """读取已经过滤后的数据"""
# filtered_data_path = 'E:/Python/MI_Analysis/data_figures/filtered_data/filtered_data.xlsx'
# 
# print("正在读取已过滤的数据...")
# data_filtered = {}
# with pd.ExcelFile(filtered_data_path) as xls:
#     for sheet_name in xls.sheet_names:
#         data_filtered[sheet_name] = pd.read_excel(xls, sheet_name=sheet_name, index_col=0)
#         print(f"已读取 {sheet_name}: {data_filtered[sheet_name].shape} (特征×样本)")
# 
# """确保样本一致性并提取共同样本"""
# common_samples = list(data_filtered['Bacteria'].columns)
# for domain, df in data_filtered.items():
#     common_samples = list(set(common_samples) & set(df.columns))
# 
# print(f"\n共同样本数量: {len(common_samples)}")
# 
# """根据共同样本过滤每个数据集，并确保样本顺序一致"""
# for domain in data_filtered.keys():
#     data_filtered[domain] = data_filtered[domain][common_samples]
# 
# """创建样本元数据（从样本名提取分组信息）"""
# sample_metadata = pd.DataFrame(index=common_samples)
# sample_metadata['Group'] = ['Control' if sample.startswith('CON') else 'MI' for sample in common_samples]
# print("\n样本分组分布:")
# print(sample_metadata['Group'].value_counts())
# 
# """根据您的七种方案定义差异分析函数"""
# def run_differential_analysis_scheme(df, metadata, domain, scheme_name):
#     """
#     根据七种特定方案进行差异分析
#     """
#     control_samples = metadata[metadata['Group'] == 'Control'].index
#     mi_samples = metadata[metadata['Group'] == 'MI'].index
#     
#     results = []
#     p_values = []
#     
#     for feature in df.index:
#         control_vals = df.loc[feature, control_samples].values
#         mi_vals = df.loc[feature, mi_samples].values
#         
#         # 计算log2FC
#         control_mean = np.mean(control_vals)
#         mi_mean = np.mean(mi_vals)
#         log2fc = np.log2((mi_mean + 1e-10) / (control_mean + 1e-10))
#         
#         # 只有当两组都有非零值时才进行检验
#         if np.any(control_vals > 0) and np.any(mi_vals > 0):
#             try:
#                 stat, p_val = mannwhitneyu(control_vals, mi_vals, alternative='two-sided')
#             except:
#                 p_val = 1.0
#         else:
#             p_val = 1.0
#         
#         results.append({
#             'feature': feature,
#             'log2fc': log2fc,
#             'p_value': p_val,
#             'control_mean': control_mean,
#             'mi_mean': mi_mean
#         })
#         p_values.append(p_val)
#     
#     # 转换为DataFrame
#     result_df = pd.DataFrame(results)
#     
#     # FDR校正
#     reject, fdr_values, _, _ = multipletests(p_values, method='fdr_bh')
#     result_df['fdr'] = fdr_values
#     
#     # 根据不同的方案应用不同的筛选标准
#     if scheme_name == 'Scheme1':
#         result_df['significant'] = (
#             (result_df['fdr'] < 0.05) & 
#             (result_df['p_value'] < 0.05) & 
#             (abs(result_df['log2fc']) > 1)
#         )
#     
#     elif scheme_name == 'Scheme2':
#         result_df['significant'] = (
#             (result_df['fdr'] < 0.2) & 
#             (result_df['p_value'] < 0.2) & 
#             (abs(result_df['log2fc']) > 0.7)
#         )
#     
#     elif scheme_name == 'Scheme3':
#         result_df['significant'] = (
#             (result_df['fdr'] < 0.2) & 
#             (result_df['p_value'] < 0.2) & 
#             (abs(result_df['log2fc']) > 0.3)
#         )
#     
#     elif scheme_name == 'Scheme4':
#         result_df['significant'] = (
#             (result_df['fdr'] < 0.2) & 
#             (abs(result_df['log2fc']) > 0.5)
#         )
#     
#     elif scheme_name == 'Scheme5':
#         result_df['significant'] = (
#             (result_df['fdr'] < 0.2) & 
#             (abs(result_df['log2fc']) > 0.7)
#         )
#     
#     elif scheme_name == 'Scheme6':
#         if domain == 'Archaea':
#             result_df['significant'] = (
#                 (result_df['p_value'] < 0.05) & 
#                 (abs(result_df['log2fc']) > 0.7)
#             )
#         else:
#             result_df['significant'] = (
#                 (result_df['fdr'] < 0.2) & 
#                 (abs(result_df['log2fc']) > 0.7)
#             )
#     
#     elif scheme_name == 'Scheme7':
#         if domain == 'Archaea':
#             result_df['significant'] = (
#                 (result_df['p_value'] < 0.1) & 
#                 (abs(result_df['log2fc']) > 0.7)
#             )
#         else:
#             result_df['significant'] = (
#                 (result_df['fdr'] < 0.2) & 
#                 (abs(result_df['log2fc']) > 0.7)
#             )
#     
#     significant_features = result_df[result_df['significant']]['feature'].tolist()
#     
#     return significant_features, result_df
# 
# def evaluate_scheme_performance(scheme_name, sig_features, full_results, df_domain, metadata):
#     """
#     综合评估方案性能
#     """
#     if len(sig_features) == 0:
#         return {
#             'num_features': 0,
#             'r_squared': 0,
#             'p_value': 1,
#             'mean_abs_logfc': 0,
#             'stability_score': 0,
#             'biological_plausibility': 0,
#             'composite_score': 0
#         }
#     
#     # 1. 群落解释度 (R²) - 最重要指标
#     df_subset = df_domain.loc[sig_features]
#     bc_dm_subset = beta_diversity('braycurtis', df_subset.T.values, ids=df_subset.T.index)
#     permanova_result = permanova(bc_dm_subset, metadata['Group'])
#     r_squared = permanova_result['test statistic']
#     
#     # 2. 效应大小
#     mean_abs_logfc = full_results[full_results['significant']]['log2fc'].abs().mean()
#     
#     # 3. 特征数量合理性 (避免过多或过少)
#     total_features = len(df_domain)
#     feature_ratio = len(sig_features) / total_features
#     # 理想范围：5%-20%的特征被选中
#     if feature_ratio < 0.05:
#         quantity_score = feature_ratio / 0.05  # 惩罚过少
#     elif feature_ratio > 0.2:
#         quantity_score = 0.2 / feature_ratio  # 惩罚过多
#     else:
#         quantity_score = 1.0  # 理想范围
#     
#     # 4. 稳定性评分 (基于FDR/p值的严格程度)
#     if 'fdr' in scheme_name.lower() or scheme_name in ['Scheme1', 'Scheme2', 'Scheme3', 'Scheme4', 'Scheme5']:
#         # 使用FDR的方案更稳定
#         stability_score = 0.8
#     else:
#         # 使用p值的方案相对不稳定
#         stability_score = 0.6
#     
#     # 5. 生物学合理性 (基于效应大小)
#     if mean_abs_logfc > 1.5:
#         bio_score = 1.0  # 大效应，生物学意义明确
#     elif mean_abs_logfc > 1.0:
#         bio_score = 0.8  # 中等效应
#     elif mean_abs_logfc > 0.5:
#         bio_score = 0.6  # 小效应
#     else:
#         bio_score = 0.3  # 效应太小
#     
#     # 综合评分
#     composite_score = (
#         r_squared * 0.4 +           # 群落解释度最重要
#         quantity_score * 0.2 +       # 特征数量合理性
#         stability_score * 0.2 +      # 方法稳定性
#         bio_score * 0.2              # 生物学合理性
#     )
#     
#     return {
#         'num_features': len(sig_features),
#         'r_squared': r_squared,
#         'p_value': permanova_result['p-value'],
#         'mean_abs_logfc': mean_abs_logfc,
#         'stability_score': stability_score,
#         'biological_plausibility': bio_score,
#         'composite_score': composite_score
#     }
# 
# ###############################################################################
# #                         第一阶段：策略二（按类别分析）                         #
# ###############################################################################
# 
# print("\n" + "="*80)
# print("第一阶段：策略二 - 按类别分析")
# print("="*80)
# 
# permanova_results = {}
# 
# for domain, df in data_filtered.items():
#     print(f"\n=== 正在分析 {domain} 的整体群落结构 ===")
#     
#     try:
#         bc_dm = beta_diversity('braycurtis', df.T.values, ids=df.T.index)
#         permanova_result = permanova(bc_dm, sample_metadata['Group'])
#         permanova_results[domain] = permanova_result
#         
#         print(f"{domain} PERMANOVA结果:")
#         print(f"  R²: {permanova_result['test statistic']:.4f}")
#         print(f"  p-value: {permanova_result['p-value']:.4f}")
#         
#     except Exception as e:
#         print(f"  分析{domain}时出错: {e}")
#         permanova_results[domain] = {'test statistic': 0, 'p-value': 1}
# 
# # 比较各类别结果
# df_permanova_comparison = pd.DataFrame.from_dict(permanova_results, orient='index')
# df_permanova_comparison = df_permanova_comparison[['test statistic', 'p-value']]
# df_permanova_comparison.columns = ['R_squared', 'p_value']
# df_permanova_comparison = df_permanova_comparison.sort_values('R_squared', ascending=False)
# 
# print("\n=== 各类别PERMANOVA结果比较 ===")
# print(df_permanova_comparison)
# 
# # 确定核心关注类别
# core_domains = df_permanova_comparison[
#     (df_permanova_comparison['p_value'] < 0.05) & 
#     (df_permanova_comparison['R_squared'] > 0.05)
# ].index.tolist()
# 
# if not core_domains:
#     core_domains = df_permanova_comparison[
#         df_permanova_comparison['p_value'] < 0.1
#     ].index.tolist()
# 
# print(f"\n建议重点关注以下类别: {core_domains}")
# 
# ###############################################################################
# #                         第二阶段：策略一（按方案分析）                         #
# ###############################################################################
# 
# print("\n" + "="*80)
# print("第二阶段：策略一 - 七种方案综合评估")
# print("="*80)
# 
# # 定义七种方案
# schemes = {
#     'Scheme1': 'FDR、P-value阈值设置0.05，Log2FC阈值设置1',
#     'Scheme2': 'FDR、P-value阈值设置0.2，Log2FC阈值设置0.7', 
#     'Scheme3': 'FDR、P-value阈值设置0.2，Log2FC阈值设置0.3',
#     'Scheme4': 'FDR阈值设置0.2，Log2FC阈值设置0.5',
#     'Scheme5': 'FDR阈值设置0.2，Log2FC阈值设置0.7',
#     'Scheme6': '非古菌FDR阈值设置0.2，Log2FC阈值设置0.7，古菌P_value阈值设置0.05',
#     'Scheme7': '非古菌FDR阈值设置0.2，Log2FC阈值设置0.7，古菌P_value阈值设置0.1'
# }
# 
# scheme_evaluation = {}
# detailed_results = {}
# final_recommendations = {}
# 
# for domain in data_filtered.keys():  # 对所有类别进行评估，不限于core_domains
#     print(f"\n=== 评估 {domain} 的七种方案 ===")
#     df_domain = data_filtered[domain]
#     scheme_evaluation[domain] = {}
#     detailed_results[domain] = {}
#     
#     domain_scheme_scores = []
#     
#     for scheme_name, scheme_desc in schemes.items():
#         print(f"\n方案: {scheme_name} - {scheme_desc}")
#         
#         try:
#             sig_features, full_results = run_differential_analysis_scheme(
#                 df_domain, sample_metadata, domain, scheme_name
#             )
#             detailed_results[domain][scheme_name] = full_results
#             
#             # 评估方案性能
#             performance = evaluate_scheme_performance(
#                 scheme_name, sig_features, full_results, df_domain, sample_metadata
#             )
#             scheme_evaluation[domain][scheme_name] = performance
#             
#             domain_scheme_scores.append((scheme_name, performance['composite_score']))
#             
#             print(f"  显著特征: {performance['num_features']}个")
#             print(f"  群落解释度 R²: {performance['r_squared']:.4f}")
#             print(f"  平均|log2FC|: {performance['mean_abs_logfc']:.2f}")
#             print(f"  综合评分: {performance['composite_score']:.3f}")
#             
#         except Exception as e:
#             print(f"  方案执行失败: {e}")
#             scheme_evaluation[domain][scheme_name] = {
#                 'num_features': 0, 'r_squared': 0, 'composite_score': 0
#             }
#     
#     # 为该类别选择最佳方案
#     if domain_scheme_scores:
#         best_scheme = max(domain_scheme_scores, key=lambda x: x[1])
#         final_recommendations[domain] = {
#             'best_scheme': best_scheme[0],
#             'best_score': best_scheme[1],
#             'scheme_description': schemes[best_scheme[0]]
#         }
# 
# # 保存详细结果
# print("\n正在保存详细结果...")
# output_path = "E:/Python/MI_Analysis/data_figures/global_integrated/final_scheme_recommendations.xlsx"
# with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
#     # 保存每个类别的详细评估结果
#     for domain, schemes_dict in scheme_evaluation.items():
#         domain_results = []
#         for scheme_name, performance in schemes_dict.items():
#             performance['scheme'] = scheme_name
#             performance['domain'] = domain
#             domain_results.append(performance)
#         
#         df_domain = pd.DataFrame(domain_results)
#         df_domain.to_excel(writer, sheet_name=domain[:31], index=False)
#     
#     # 保存最终推荐结果
#     final_df = pd.DataFrame([
#         {
#             'Domain': domain,
#             'Recommended_Scheme': rec['best_scheme'],
#             'Composite_Score': rec['best_score'],
#             'Scheme_Description': rec['scheme_description']
#         }
#         for domain, rec in final_recommendations.items()
#     ])
#     final_df.to_excel(writer, sheet_name='Final_Recommendations', index=False)
# 
# print(f"详细结果已保存至: {output_path}")
# 
# ###############################################################################
# #                         最终推荐结果                         #
# ###############################################################################
# 
# print("\n" + "="*80)
# print("最终方案推荐结果")
# print("="*80)
# 
# print("\n🌟 最佳方案推荐:")
# print("-" * 60)
# 
# for domain, recommendation in final_recommendations.items():
#     best_scheme = recommendation['best_scheme']
#     best_score = recommendation['best_score']
#     scheme_desc = recommendation['scheme_description']
#     
#     performance = scheme_evaluation[domain][best_scheme]
#     
#     print(f"\n📊 {domain}:")
#     print(f"  推荐方案: {best_scheme}")
#     print(f"  方案描述: {scheme_desc}")
#     print(f"  综合评分: {best_score:.3f}")
#     print(f"  显著特征数: {performance['num_features']}")
#     print(f"  群落解释度 R²: {performance['r_squared']:.4f}")
#     print(f"  平均效应大小: |log2FC| = {performance['mean_abs_logfc']:.2f}")
# 
# # 全局最佳方案统计
# print("\n" + "="*80)
# print("全局方案使用统计")
# print("="*80)
# 
# scheme_usage = {}
# for domain, recommendation in final_recommendations.items():
#     best_scheme = recommendation['best_scheme']
#     if best_scheme in scheme_usage:
#         scheme_usage[best_scheme] += 1
#     else:
#         scheme_usage[best_scheme] = 1
# 
# print("各方案被推荐次数:")
# for scheme, count in sorted(scheme_usage.items(), key=lambda x: x[1], reverse=True):
#     print(f"  {scheme}: {count}次")
# 
# # 找出最常用的方案
# if scheme_usage:
#     most_common_scheme = max(scheme_usage.items(), key=lambda x: x[1])
#     print(f"\n🏆 全局最推荐方案: {most_common_scheme[0]} (被推荐{most_common_scheme[1]}次)")
#     print(f"   方案描述: {schemes[most_common_scheme[0]]}")
# 
# print("\n" + "="*80)
# print("分析完成！")
# print("="*80)

正在读取已过滤的数据...
已读取 Bacteria: (335, 85) (特征×样本)
已读取 Fungi: (194, 85) (特征×样本)
已读取 Virus: (152, 85) (特征×样本)
已读取 Archaea: (365, 85) (特征×样本)

共同样本数量: 85

样本分组分布:
Group
Control    47
MI         38
Name: count, dtype: int64


In [11]:
import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu
from skbio.diversity import beta_diversity
from skbio.stats.distance import permanova
from statsmodels.stats.multitest import multipletests

"""设置显示选项"""
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

"""读取已经过滤后的数据"""
filtered_data_path = 'E:/Python/MI_Analysis/data_figures/filtered_data/filtered_data.xlsx'

print("正在读取已过滤的数据...")
data_filtered = {}
with pd.ExcelFile(filtered_data_path) as xls:
    for sheet_name in xls.sheet_names:
        data_filtered[sheet_name] = pd.read_excel(xls, sheet_name=sheet_name, index_col=0)
        print(f"已读取 {sheet_name}: {data_filtered[sheet_name].shape} (特征×样本)")

"""确保样本一致性并提取共同样本"""
common_samples = list(data_filtered['Bacteria'].columns)
for domain, df in data_filtered.items():
    common_samples = list(set(common_samples) & set(df.columns))

print(f"\n共同样本数量: {len(common_samples)}")

"""根据共同样本过滤每个数据集，并确保样本顺序一致"""
for domain in data_filtered.keys():
    data_filtered[domain] = data_filtered[domain][common_samples]

"""创建样本元数据（从样本名提取分组信息）"""
sample_metadata = pd.DataFrame(index=common_samples)
sample_metadata['Group'] = ['Control' if sample.startswith('CON') else 'MI' for sample in common_samples]
print("\n样本分组分布:")
print(sample_metadata['Group'].value_counts())

"""根据您的七种方案定义差异分析函数"""
def run_differential_analysis_scheme(df, metadata, domain, scheme_name):
    """
    根据七种特定方案进行差异分析
    """
    control_samples = metadata[metadata['Group'] == 'Control'].index
    mi_samples = metadata[metadata['Group'] == 'MI'].index
    
    results = []
    p_values = []
    
    for feature in df.index:
        control_vals = df.loc[feature, control_samples].values
        mi_vals = df.loc[feature, mi_samples].values
        
        # 计算log2FC
        control_mean = np.mean(control_vals)
        mi_mean = np.mean(mi_vals)
        log2fc = np.log2((mi_mean + 1e-10) / (control_mean + 1e-10))
        
        # 只有当两组都有非零值时才进行检验
        if np.any(control_vals > 0) and np.any(mi_vals > 0):
            try:
                stat, p_val = mannwhitneyu(control_vals, mi_vals, alternative='two-sided')
            except:
                p_val = 1.0
        else:
            p_val = 1.0
        
        results.append({
            'feature': feature,
            'log2fc': log2fc,
            'p_value': p_val,
            'control_mean': control_mean,
            'mi_mean': mi_mean
        })
        p_values.append(p_val)
    
    # 转换为DataFrame
    result_df = pd.DataFrame(results)
    
    # FDR校正
    reject, fdr_values, _, _ = multipletests(p_values, method='fdr_bh')
    result_df['fdr'] = fdr_values
    
    # 根据不同的方案应用不同的筛选标准
    if scheme_name == 'Scheme1':
        result_df['significant'] = (
            (result_df['fdr'] < 0.05) & 
            (result_df['p_value'] < 0.05) & 
            (abs(result_df['log2fc']) > 1)
        )
    
    elif scheme_name == 'Scheme2':
        result_df['significant'] = (
            (result_df['fdr'] < 0.2) & 
            (result_df['p_value'] < 0.2) & 
            (abs(result_df['log2fc']) > 0.7)
        )
    
    elif scheme_name == 'Scheme3':
        result_df['significant'] = (
            (result_df['fdr'] < 0.2) & 
            (result_df['p_value'] < 0.2) & 
            (abs(result_df['log2fc']) > 0.3)
        )
    
    elif scheme_name == 'Scheme4':
        result_df['significant'] = (
            (result_df['fdr'] < 0.2) & 
            (abs(result_df['log2fc']) > 0.5)
        )
    
    elif scheme_name == 'Scheme5':
        result_df['significant'] = (
            (result_df['fdr'] < 0.2) & 
            (abs(result_df['log2fc']) > 0.7)
        )
    
    elif scheme_name == 'Scheme6':
        if domain == 'Archaea':
            result_df['significant'] = (
                (result_df['p_value'] < 0.05) & 
                (abs(result_df['log2fc']) > 0.7)
            )
        else:
            result_df['significant'] = (
                (result_df['fdr'] < 0.2) & 
                (abs(result_df['log2fc']) > 0.7)
            )
    
    elif scheme_name == 'Scheme7':
        if domain == 'Archaea':
            result_df['significant'] = (
                (result_df['p_value'] < 0.1) & 
                (abs(result_df['log2fc']) > 0.7)
            )
        else:
            result_df['significant'] = (
                (result_df['fdr'] < 0.2) & 
                (abs(result_df['log2fc']) > 0.7)
            )
    
    significant_features = result_df[result_df['significant']]['feature'].tolist()
    
    return significant_features, result_df

def evaluate_scheme_performance(domain, scheme_name, sig_features, full_results, df_domain, metadata):
    """
    综合评估方案性能 - 已添加完整的错误处理
    """
    # 初始化为默认值（避免NaN）
    default_result = {
        'num_features': 0,
        'r_squared': 0,
        'p_value': 1,
        'mean_abs_logfc': 0,
        'stability_score': 0,
        'biological_plausibility': 0,
        'composite_score': 0
    }
    
    # 检查是否有显著特征
    if len(sig_features) == 0:
        return default_result
    
    try:
        # 1. 群落解释度 (R²) - 最重要指标
        df_subset = df_domain.loc[sig_features]
        
        # 检查数据有效性
        if df_subset.empty or df_subset.shape[0] == 0:
            print(f"警告: {domain}的{scheme_name}方案筛选出的特征为空")
            return default_result
        
        # 检查数据是否全为0
        if df_subset.sum().sum() == 0:
            print(f"警告: {domain}的{scheme_name}方案筛选出的特征全为0")
            return default_result
            
        bc_dm_subset = beta_diversity('braycurtis', df_subset.T.values, ids=df_subset.T.index)
        permanova_result = permanova(bc_dm_subset, metadata['Group'])
        
        # 检查PERMANOVA结果有效性
        r_squared = permanova_result['test statistic']
        p_value = permanova_result['p-value']
        
        if np.isnan(r_squared) or np.isinf(r_squared):
            print(f"警告: {domain}的{scheme_name}方案R²值为无效值: {r_squared}，使用默认值0")
            r_squared = 0
        if np.isnan(p_value) or np.isinf(p_value):
            p_value = 1
        
        # 2. 效应大小
        sig_results = full_results[full_results['significant']]
        if len(sig_results) > 0:
            mean_abs_logfc = sig_results['log2fc'].abs().mean()
            if np.isnan(mean_abs_logfc) or np.isinf(mean_abs_logfc):
                mean_abs_logfc = 0
        else:
            mean_abs_logfc = 0
        
        # 3. 特征数量合理性 (避免过多或过少)
        total_features = len(df_domain)
        feature_ratio = len(sig_features) / total_features
        # 理想范围：5%-20%的特征被选中
        if feature_ratio < 0.05:
            quantity_score = feature_ratio / 0.05  # 惩罚过少
        elif feature_ratio > 0.2:
            quantity_score = 0.2 / feature_ratio  # 惩罚过多
        else:
            quantity_score = 1.0  # 理想范围
        
        # 4. 稳定性评分 (基于FDR/p值的严格程度)
        if 'fdr' in scheme_name.lower() or scheme_name in ['Scheme1', 'Scheme2', 'Scheme3', 'Scheme4', 'Scheme5']:
            # 使用FDR的方案更稳定
            stability_score = 0.8
        else:
            # 使用p值的方案相对不稳定
            stability_score = 0.6
        
        # 5. 生物学合理性 (基于效应大小)
        if mean_abs_logfc > 1.5:
            bio_score = 1.0  # 大效应，生物学意义明确
        elif mean_abs_logfc > 1.0:
            bio_score = 0.8  # 中等效应
        elif mean_abs_logfc > 0.5:
            bio_score = 0.6  # 小效应
        else:
            bio_score = 0.3  # 效应太小
        
        # 综合评分
        composite_score = (
            r_squared * 0.4 +           # 群落解释度最重要
            quantity_score * 0.2 +       # 特征数量合理性
            stability_score * 0.2 +      # 方法稳定性
            bio_score * 0.2              # 生物学合理性
        )
        
        # 确保综合评分有效
        if np.isnan(composite_score) or np.isinf(composite_score):
            composite_score = 0
        
        return {
            'num_features': len(sig_features),
            'r_squared': r_squared,
            'p_value': p_value,
            'mean_abs_logfc': mean_abs_logfc,
            'stability_score': stability_score,
            'biological_plausibility': bio_score,
            'composite_score': composite_score
        }
        
    except Exception as e:
        print(f"警告: {domain}的{scheme_name}方案评估失败: {e}")
        return default_result

正在读取已过滤的数据...
已读取 Bacteria: (335, 85) (特征×样本)
已读取 Fungi: (194, 85) (特征×样本)
已读取 Virus: (152, 85) (特征×样本)
已读取 Archaea: (365, 85) (特征×样本)

共同样本数量: 85

样本分组分布:
Group
Control    47
MI         38
Name: count, dtype: int64


In [12]:
###############################################################################
#                         第一阶段：确定对心梗影响最大的微生物类别               #
###############################################################################

print("\n" + "="*80)
print("第一阶段：确定对心梗影响最大的微生物类别")
print("="*80)

permanova_results = {}

for domain, df in data_filtered.items():
    print(f"\n=== 正在分析 {domain} 的整体群落结构 ===")
    
    try:
        bc_dm = beta_diversity('braycurtis', df.T.values, ids=df.T.index)
        permanova_result = permanova(bc_dm, sample_metadata['Group'])
        permanova_results[domain] = permanova_result
        
        print(f"{domain} PERMANOVA结果:")
        print(f"  R²: {permanova_result['test statistic']:.4f}")
        print(f"  p-value: {permanova_result['p-value']:.4f}")
        
    except Exception as e:
        print(f"  分析{domain}时出错: {e}")
        permanova_results[domain] = {'test statistic': 0, 'p-value': 1}

# 比较各类别结果
df_permanova_comparison = pd.DataFrame.from_dict(permanova_results, orient='index')
df_permanova_comparison = df_permanova_comparison[['test statistic', 'p-value']]
df_permanova_comparison.columns = ['R_squared', 'p_value']
df_permanova_comparison = df_permanova_comparison.sort_values('R_squared', ascending=False)

print("\n=== 各类别PERMANOVA结果比较（按R²排序）===")
print(df_permanova_comparison)

# 确定对心梗影响最大的微生物类别
# 选择标准：p_value < 0.05 且 R²最大（群落解释度最高）
significant_domains = df_permanova_comparison[df_permanova_comparison['p_value'] < 0.05]

if len(significant_domains) > 0:
    # 选择R²最大的显著类别
    most_impactful_domain = significant_domains.iloc[0].name
    most_impactful_r2 = significant_domains.iloc[0]['R_squared']
    most_impactful_p = significant_domains.iloc[0]['p_value']
    
    print(f"\n🎯 确定对心梗影响最大的微生物类别: {most_impactful_domain}")
    print(f"  群落解释度 R²: {most_impactful_r2:.4f}")
    print(f"  显著性 p-value: {most_impactful_p:.4f}")
    
else:
    # 如果没有显著类别，选择p值最小且R²最大的类别
    most_impactful_domain = df_permanova_comparison.iloc[0].name
    most_impactful_r2 = df_permanova_comparison.iloc[0]['R_squared']
    most_impactful_p = df_permanova_comparison.iloc[0]['p_value']
    
    print(f"\n⚠️ 没有发现显著差异的微生物类别，选择影响相对最大的类别: {most_impactful_domain}")
    print(f"  群落解释度 R²: {most_impactful_r2:.4f}")
    print(f"  显著性 p-value: {most_impactful_p:.4f}")


第一阶段：确定对心梗影响最大的微生物类别

=== 正在分析 Bacteria 的整体群落结构 ===
Bacteria PERMANOVA结果:
  R²: 1.9897
  p-value: 0.0490

=== 正在分析 Fungi 的整体群落结构 ===
Fungi PERMANOVA结果:
  R²: 1.2133
  p-value: 0.2630

=== 正在分析 Virus 的整体群落结构 ===
Virus PERMANOVA结果:
  R²: 4.7234
  p-value: 0.0030

=== 正在分析 Archaea 的整体群落结构 ===
Archaea PERMANOVA结果:
  R²: 2.6550
  p-value: 0.0590

=== 各类别PERMANOVA结果比较（按R²排序）===
          R_squared  p_value
Virus      4.723423    0.003
Archaea    2.654952    0.059
Bacteria   1.989671    0.049
Fungi      1.213263    0.263

🎯 确定对心梗影响最大的微生物类别: Virus
  群落解释度 R²: 4.7234
  显著性 p-value: 0.0030


In [13]:
###############################################################################
#             第二阶段：对影响最大的微生物类别进行七种方案验证                   #
###############################################################################

print("\n" + "="*80)
print(f"第二阶段：对 {most_impactful_domain} 进行七种方案验证")
print("="*80)

# 定义七种方案
schemes = {
    'Scheme1': 'FDR、P-value阈值设置0.05，Log2FC阈值设置1',
    'Scheme2': 'FDR、P-value阈值设置0.2，Log2FC阈值设置0.7', 
    'Scheme3': 'FDR、P-value阈值设置0.2，Log2FC阈值设置0.3',
    'Scheme4': 'FDR阈值设置0.2，Log2FC阈值设置0.5',
    'Scheme5': 'FDR阈值设置0.2，Log2FC阈值设置0.7',
    'Scheme6': '非古菌FDR阈值设置0.2，Log2FC阈值设置0.7，古菌P_value阈值设置0.05',
    'Scheme7': '非古菌FDR阈值设置0.2，Log2FC阈值设置0.7，古菌P_value阈值设置0.1'
}

scheme_evaluation = {}
detailed_results = {}

print(f"\n=== 对 {most_impactful_domain} 进行七种方案评估 ===")
df_domain = data_filtered[most_impactful_domain]

scheme_scores = []

for scheme_name, scheme_desc in schemes.items():
    print(f"\n🔍 评估方案: {scheme_name} - {scheme_desc}")
    
    try:
        sig_features, full_results = run_differential_analysis_scheme(
            df_domain, sample_metadata, most_impactful_domain, scheme_name
        )
        detailed_results[scheme_name] = full_results
        
        # 评估方案性能
        performance = evaluate_scheme_performance(
            most_impactful_domain, scheme_name, sig_features, full_results, df_domain, sample_metadata
        )
        scheme_evaluation[scheme_name] = performance
        
        scheme_scores.append((scheme_name, performance['composite_score']))
        
        print(f"  ✅ 显著特征: {performance['num_features']}个")
        print(f"  ✅ 群落解释度 R²: {performance['r_squared']:.4f}")
        print(f"  ✅ 平均|log2FC|: {performance['mean_abs_logfc']:.2f}")
        print(f"  ✅ 综合评分: {performance['composite_score']:.3f}")
        
    except Exception as e:
        print(f"  ❌ 方案执行失败: {e}")
        scheme_evaluation[scheme_name] = {
            'num_features': 0, 'r_squared': 0, 'p_value': 1, 'mean_abs_logfc': 0,
            'stability_score': 0, 'biological_plausibility': 0, 'composite_score': 0
        }

# 选择最佳方案
if scheme_scores:
    best_scheme = max(scheme_scores, key=lambda x: x[1])
    best_scheme_name = best_scheme[0]
    best_scheme_score = best_scheme[1]
    
    print(f"\n🎉 为 {most_impactful_domain} 确定的最佳方案: {best_scheme_name}")
    print(f"   综合评分: {best_scheme_score:.3f}")
    print(f"   方案描述: {schemes[best_scheme_name]}")
    
    # 显示最佳方案的详细结果
    best_performance = scheme_evaluation[best_scheme_name]
    print(f"\n📊 最佳方案详细性能:")
    print(f"   显著特征数: {best_performance['num_features']}")
    print(f"   群落解释度 R²: {best_performance['r_squared']:.4f}")
    print(f"   PERMANOVA p-value: {best_performance['p_value']:.4f}")
    print(f"   平均效应大小: |log2FC| = {best_performance['mean_abs_logfc']:.2f}")
    print(f"   稳定性评分: {best_performance['stability_score']:.1f}")
    print(f"   生物学合理性: {best_performance['biological_plausibility']:.1f}")

# 保存详细结果
print("\n正在保存详细结果...")
output_path = "E:/Python/MI_Analysis/data_figures/global_integrated/focused_scheme_analysis.xlsx"
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    # 保存七种方案的比较结果
    scheme_comparison = []
    for scheme_name, performance in scheme_evaluation.items():
        performance['scheme'] = scheme_name
        performance['domain'] = most_impactful_domain
        performance['scheme_description'] = schemes[scheme_name]
        scheme_comparison.append(performance)
    
    df_comparison = pd.DataFrame(scheme_comparison)
    # 按综合评分排序
    df_comparison = df_comparison.sort_values('composite_score', ascending=False)
    df_comparison.to_excel(writer, sheet_name='Scheme_Comparison', index=False)
    
    # 保存最佳方案的详细差异分析结果
    if best_scheme_name in detailed_results:
        detailed_results[best_scheme_name].to_excel(
            writer, sheet_name=f'Best_Scheme_Details', index=False
        )
    
    # 保存第一阶段结果
    df_permanova_comparison.to_excel(writer, sheet_name='Domain_Comparison')

print(f"详细结果已保存至: {output_path}")


第二阶段：对 Virus 进行七种方案验证

=== 对 Virus 进行七种方案评估 ===

🔍 评估方案: Scheme1 - FDR、P-value阈值设置0.05，Log2FC阈值设置1
警告: Virus的Scheme1方案R²值为无效值: nan，使用默认值0
  ✅ 显著特征: 5个
  ✅ 群落解释度 R²: 0.0000
  ✅ 平均|log2FC|: 2.59
  ✅ 综合评分: 0.492

🔍 评估方案: Scheme2 - FDR、P-value阈值设置0.2，Log2FC阈值设置0.7
警告: Virus的Scheme2方案R²值为无效值: nan，使用默认值0
  ✅ 显著特征: 8个
  ✅ 群落解释度 R²: 0.0000
  ✅ 平均|log2FC|: 3.10
  ✅ 综合评分: 0.560

🔍 评估方案: Scheme3 - FDR、P-value阈值设置0.2，Log2FC阈值设置0.3
  ✅ 显著特征: 9个
  ✅ 群落解释度 R²: 10.1775
  ✅ 平均|log2FC|: 2.79
  ✅ 综合评分: 4.631

🔍 评估方案: Scheme4 - FDR阈值设置0.2，Log2FC阈值设置0.5
警告: Virus的Scheme4方案R²值为无效值: nan，使用默认值0
  ✅ 显著特征: 8个
  ✅ 群落解释度 R²: 0.0000
  ✅ 平均|log2FC|: 3.10
  ✅ 综合评分: 0.560

🔍 评估方案: Scheme5 - FDR阈值设置0.2，Log2FC阈值设置0.7
警告: Virus的Scheme5方案R²值为无效值: nan，使用默认值0
  ✅ 显著特征: 8个
  ✅ 群落解释度 R²: 0.0000
  ✅ 平均|log2FC|: 3.10
  ✅ 综合评分: 0.560

🔍 评估方案: Scheme6 - 非古菌FDR阈值设置0.2，Log2FC阈值设置0.7，古菌P_value阈值设置0.05
警告: Virus的Scheme6方案R²值为无效值: nan，使用默认值0
  ✅ 显著特征: 8个
  ✅ 群落解释度 R²: 0.0000
  ✅ 平均|log2FC|: 3.10
  ✅ 综合评分: 0.520

🔍 评估方案: Scheme7 - 非古

In [14]:
###############################################################################
#                         最终推荐结果                         #
###############################################################################

print("\n" + "="*80)
print("最终方案推荐结果")
print("="*80)

print(f"\n🌟 基于两阶段分析的最佳推荐:")
print("-" * 50)

print(f"\n📈 第一阶段 - 微生物类别筛选:")
print(f"   对心梗影响最大的类别: {most_impactful_domain}")
print(f"   群落解释度 R²: {most_impactful_r2:.4f}")
print(f"   显著性 p-value: {most_impactful_p:.4f}")

print(f"\n🎯 第二阶段 - 差异分析方案优化:")
print(f"   最佳方案: {best_scheme_name}")
print(f"   方案描述: {schemes[best_scheme_name]}")
print(f"   综合评分: {best_scheme_score:.3f}")

print(f"\n💡 推荐使用建议:")
print(f"   在对 {most_impactful_domain} 进行心梗相关的差异分析时，")
print(f"   推荐采用 {best_scheme_name} 方案，该方案在综合评估中表现最佳。")

# 显示所有方案的排名
print(f"\n🏆 七种方案综合评分排名:")
print("-" * 40)

ranked_schemes = sorted(scheme_scores, key=lambda x: x[1], reverse=True)
for i, (scheme, score) in enumerate(ranked_schemes, 1):
    print(f"  {i}. {scheme}: {score:.3f} - {schemes[scheme]}")

print("\n" + "="*80)
print("分析完成！")
print("="*80)


最终方案推荐结果

🌟 基于两阶段分析的最佳推荐:
--------------------------------------------------

📈 第一阶段 - 微生物类别筛选:
   对心梗影响最大的类别: Virus
   群落解释度 R²: 4.7234
   显著性 p-value: 0.0030

🎯 第二阶段 - 差异分析方案优化:
   最佳方案: Scheme3
   方案描述: FDR、P-value阈值设置0.2，Log2FC阈值设置0.3
   综合评分: 4.631

💡 推荐使用建议:
   在对 Virus 进行心梗相关的差异分析时，
   推荐采用 Scheme3 方案，该方案在综合评估中表现最佳。

🏆 七种方案综合评分排名:
----------------------------------------
  1. Scheme3: 4.631 - FDR、P-value阈值设置0.2，Log2FC阈值设置0.3
  2. Scheme2: 0.560 - FDR、P-value阈值设置0.2，Log2FC阈值设置0.7
  3. Scheme4: 0.560 - FDR阈值设置0.2，Log2FC阈值设置0.5
  4. Scheme5: 0.560 - FDR阈值设置0.2，Log2FC阈值设置0.7
  5. Scheme6: 0.520 - 非古菌FDR阈值设置0.2，Log2FC阈值设置0.7，古菌P_value阈值设置0.05
  6. Scheme7: 0.520 - 非古菌FDR阈值设置0.2，Log2FC阈值设置0.7，古菌P_value阈值设置0.1
  7. Scheme1: 0.492 - FDR、P-value阈值设置0.05，Log2FC阈值设置1

分析完成！
